# Project 2 - ETL Process
# Topic: Netflix Movies & TV-Shows and their Viewer Ratings
---

## Group 7: 
---
- ### Nathan Johnson 
- ### Yameena Khan 
- ### Ben Johnson 


## Project Overview
---
Our project aims to develop an analyzable dataset of the movies and tv-shows available on Netflix along with their IMBD ratings, Metascore rating, Box Office totals, movie rating (PG, PG-13, etc). 

To develop such a dataset and tables we have extracted data from two sources:
- __Netflix Movies and TV Shows CSV file obtained from Kaggle.com__ <br>
    This dataset consists of more than 8000 Netflix Movies and TVshows.
- __API requests from OMBD.com__ <br>
     We used the list of movies & tv-shows from the Netflix Movies and TV Shows CSV file obtained from Kaggle.com and ran API requests to obtain the ratings data for those movies and shows.
     
We have used Jupyter Notebook/Pandas to transform the extracted data and then used PostgreSQL to load the transformed data and then created clear and analyzable tables which can be joined and/or filtered accordingly for further analyses. 

## Importing Dependencies

In [1]:
# import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect

## Extracting Data 
---
We start by loading the CSV files into pandas dataframe in this Jupyter Notebook. First we load the Netflix Movies and TVshows dataset obtained from Kaggle and then we load the OMBD CSV that we have developed requesting data from the OMBD API website in the other Jupyter Notebook (OMDB API data extraction.ipynb) present in this Repo.

In [2]:
# this cell contains extracting the data for the first dataset
csv_netflix = 'netflix_titles.csv'
netflix_df = pd.read_csv(csv_netflix)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,25-Sep-21,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,24-Sep-21,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,24-Sep-21,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
# this cell contains extracting the data for the second dataset
# note that this csv was created utilizing an API that can be found in the repo in the file OMDB API data extraction.ipynb
csv_omdb = 'omdb_data.csv'
omdb_df = pd.read_csv(csv_omdb)
omdb_df.head()

,Title,Year,Rated,Released,Runtime,Metascore,imdbRating,Type,BoxOffice
0,Dick Johnson Is Dead,2020,PG-13,02 Oct 2020,1 min,89.0,7.4,movie,NaN
1,Blood Diamond,2006,R,08 Dec 2006,143 min,64.0,8.0,movie,"$57,377,916"
2,Ganglands,2021–,TV-MA,24 Sep 2021,2 min,NaN,7.0,series,na
3,Jailbirds New Orleans,2021–,TV-MA,24 Sep 2021,1 min,NaN,6.7,series,na
4,Kota Factory,2019–2021,TV-14,16 Apr 2019,45 min,NaN,9.1,series,na


## Transforming the Extracted Data
---

### Dataset no.1 - Netflix Movies and TV shows Dataframe

To make this Dataframe easily readible and more relevant to our goal we first selected the desired columns (show_id, type, title, rating, date it was added on netflix and the duration)

In the next step we renamed two columns to ensure that they correctly label the data in them. Converted "show_id" to just "id", as this would be used as a primary key for this dataframe and converted "date_added" to "date_added_netflix" this conveys the user that this date is not the release date of this show/movie, it is the date it was added to netflix.

In [4]:
# select columns desired for the netflix_movies_shows table that is created in PostgreSQL
netflix_df_transform = netflix_df [["show_id", "type", "title", "rating", "date_added", "duration"]]
netflix_df_transform

,show_id,type,title,rating,date_added,duration
0,s1,Movie,Dick Johnson Is Dead,PG-13,25-Sep-21,90 min
1,s2,TV Show,Blood & Water,TV-MA,24-Sep-21,2 Seasons
2,s3,TV Show,Ganglands,TV-MA,24-Sep-21,1 Season
3,s4,TV Show,Jailbirds New Orleans,TV-MA,24-Sep-21,1 Season
4,s5,TV Show,Kota Factory,TV-MA,24-Sep-21,2 Seasons
...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,R,20-Nov-19,158 min
8803,s8804,TV Show,Zombie Dumb,TV-Y7,1-Jul-19,2 Seasons
8804,s8805,Movie,Zombieland,R,1-Nov-19,88 min
8805,s8806,Movie,Zoom,PG,11-Jan-20,88 min


In [5]:
# rename columns
netflix_df_transform = netflix_df_transform.rename(columns={"show_id": "id", "date_added": "date_added_netflix"})
netflix_df_transform.head(25)

,id,type,title,rating,date_added_netflix,duration
0,s1,Movie,Dick Johnson Is Dead,PG-13,25-Sep-21,90 min
1,s2,TV Show,Blood & Water,TV-MA,24-Sep-21,2 Seasons
2,s3,TV Show,Ganglands,TV-MA,24-Sep-21,1 Season
3,s4,TV Show,Jailbirds New Orleans,TV-MA,24-Sep-21,1 Season
4,s5,TV Show,Kota Factory,TV-MA,24-Sep-21,2 Seasons
5,s6,TV Show,Midnight Mass,TV-MA,24-Sep-21,1 Season
6,s7,Movie,My Little Pony: A New Generation,PG,24-Sep-21,91 min
7,s8,Movie,Sankofa,TV-MA,24-Sep-21,125 min
8,s9,TV Show,The Great British Baking Show,TV-14,24-Sep-21,9 Seasons
9,s10,Movie,The Starling,PG-13,24-Sep-21,104 min


### Dataset no.2 - Omdb Results Dataframe

Similar to transforming Dataset no.1 first we selected the desired columns ombd dataframe we had created. We selected the Title, Released (released date),imdb rating, BoxOffice and Metascore.

Next what we did was set up an index column. The reason we did this is that this particular dataset did not have a unique value column that could be set as a primary key and we need a primary key as we intend to load this dataframe into PostgreSQL.

The following step was to rename the column names such that they are more relatable and readable. After that as we noticed the dataframe consisted of null values and NA filled in some columns (box office and Meta score). To make sure its uniform we converted all the NAs to Null values. This would make filtering easy to the user.

In [6]:
# select columns desired for the omdb_results table that is created in PostgreSQL
omdb_df_transform = omdb_df [["Title", "Released", "imdbRating", "BoxOffice", "Metascore"]]
omdb_df_transform.head(25)

,Title,Released,imdbRating,BoxOffice,Metascore
0,Dick Johnson Is Dead,02 Oct 2020,7.4,NaN,89.0
1,Blood Diamond,08 Dec 2006,8.0,"$57,377,916",64.0
2,Ganglands,24 Sep 2021,7.0,na,NaN
3,Jailbirds New Orleans,24 Sep 2021,6.7,na,NaN
4,Kota Factory,16 Apr 2019,9.1,na,NaN
5,Midnight Mass,24 Sep 2021,7.7,na,NaN
6,My Little Pony: A New Generation,24 Sep 2021,6.8,NaN,NaN
7,Sankofa,28 May 1993,7.0,"$2,198,984",93.0
8,The Great British Baking Show,27 Dec 2014,8.6,na,NaN
9,The Starling,24 Sep 2021,6.3,NaN,31.0


In [7]:
# set up a column to be the primary key column when loading toSQL
omdb_df_transform.index += 1
omdb_df_transform

,Title,Released,imdbRating,BoxOffice,Metascore
1,Dick Johnson Is Dead,02 Oct 2020,7.4,NaN,89.0
2,Blood Diamond,08 Dec 2006,8.0,"$57,377,916",64.0
3,Ganglands,24 Sep 2021,7.0,na,NaN
4,Jailbirds New Orleans,24 Sep 2021,6.7,na,NaN
5,Kota Factory,16 Apr 2019,9.1,na,NaN
...,...,...,...,...,...
7804,Zodiac,02 Mar 2007,7.7,"$33,080,084",78.0
7805,Zombie Dumb,01 Jul 2019,5.6,na,NaN
7806,Zombieland,02 Oct 2009,7.6,"$75,590,286",73.0
7807,Zoom,11 Aug 2006,4.3,"$11,989,328",26.0


In [8]:
# reset the index so there is an index for pandas purposes and a primary key column for SQL purposes
omdb_df_transform = omdb_df_transform.reset_index()
omdb_df_transform

,index,Title,Released,imdbRating,BoxOffice,Metascore
0,1,Dick Johnson Is Dead,02 Oct 2020,7.4,NaN,89.0
1,2,Blood Diamond,08 Dec 2006,8.0,"$57,377,916",64.0
2,3,Ganglands,24 Sep 2021,7.0,na,NaN
3,4,Jailbirds New Orleans,24 Sep 2021,6.7,na,NaN
4,5,Kota Factory,16 Apr 2019,9.1,na,NaN
...,...,...,...,...,...,...
7803,7804,Zodiac,02 Mar 2007,7.7,"$33,080,084",78.0
7804,7805,Zombie Dumb,01 Jul 2019,5.6,na,NaN
7805,7806,Zombieland,02 Oct 2009,7.6,"$75,590,286",73.0
7806,7807,Zoom,11 Aug 2006,4.3,"$11,989,328",26.0


In [9]:
# rename columns to match SQL table column names
omdb_df_transform = omdb_df_transform.rename(columns={"index": "id", "Title": "title", "Released": "release_date", "imdbRating": "imdb_rating", "Metascore": "meta_score", "BoxOffice": "box_office"})
omdb_df_transform

,id,title,release_date,imdb_rating,box_office,meta_score
0,1,Dick Johnson Is Dead,02 Oct 2020,7.4,NaN,89.0
1,2,Blood Diamond,08 Dec 2006,8.0,"$57,377,916",64.0
2,3,Ganglands,24 Sep 2021,7.0,na,NaN
3,4,Jailbirds New Orleans,24 Sep 2021,6.7,na,NaN
4,5,Kota Factory,16 Apr 2019,9.1,na,NaN
...,...,...,...,...,...,...
7803,7804,Zodiac,02 Mar 2007,7.7,"$33,080,084",78.0
7804,7805,Zombie Dumb,01 Jul 2019,5.6,na,NaN
7805,7806,Zombieland,02 Oct 2009,7.6,"$75,590,286",73.0
7806,7807,Zoom,11 Aug 2006,4.3,"$11,989,328",26.0


In [10]:
# replace all na text with null/NaN
omdb_df_transform['imdb_rating']=omdb_df_transform['imdb_rating'].replace('na',np.NaN)
omdb_df_transform['box_office']=omdb_df_transform['box_office'].replace('na',np.NaN)
omdb_df_transform['meta_score']=omdb_df_transform['meta_score'].replace('na',np.NaN)

In [11]:
omdb_df_transform

,id,title,release_date,imdb_rating,box_office,meta_score
0,1,Dick Johnson Is Dead,02 Oct 2020,7.4,NaN,89.0
1,2,Blood Diamond,08 Dec 2006,8.0,"$57,377,916",64.0
2,3,Ganglands,24 Sep 2021,7.0,NaN,NaN
3,4,Jailbirds New Orleans,24 Sep 2021,6.7,NaN,NaN
4,5,Kota Factory,16 Apr 2019,9.1,NaN,NaN
...,...,...,...,...,...,...
7803,7804,Zodiac,02 Mar 2007,7.7,"$33,080,084",78.0
7804,7805,Zombie Dumb,01 Jul 2019,5.6,NaN,NaN
7805,7806,Zombieland,02 Oct 2009,7.6,"$75,590,286",73.0
7806,7807,Zoom,11 Aug 2006,4.3,"$11,989,328",26.0


## Loading the Data
---
To load the data into a data base, we decided to use Postgresql. We created a database called 'etlproject_db' on our Pg_admin app. Next using SQL query we created two tables: __omdb_results & netflix_movies_shows__ with the desired column names to store the transformed data from two dataframes created above.

After creating a connection to PostgreSQL we used the inspector to confirm if the desired tables are available. Then using `df.to_sql` to load the data in dataframes to the tables.



In [12]:
# create conneciton to PostgreSQL
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'etlproject_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [13]:
 # Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

['netflix_movies_shows', 'omdb_results']

In [14]:
# load netflix dataframe into database
netflix_df_transform.to_sql(name='netflix_movies_shows', con=engine, if_exists='append', index=False)

807

In [15]:
# load omdb dataframe into database
omdb_df_transform.to_sql(name='omdb_results', con=engine, if_exists='append', index=False)

808